In [1]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://127.0.0.1:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## WorkflowTemplate <-> Pipeline & Workflow <-> Flow

WorkFlowTemplates crucially allow the definition of parameters as input variables to the entire workflow *without specifying a value*. You can also specify the default value, but it is possible to omit in cases where a sensible value just doesnt apply.

Submitting these WorkFlow templates doesnt execute anything on the ArgoWorkflow engine, but rather adds a template that future `Workflow`s can be derived from, together with configurable workflow input parameters.

This allows for the following entity mapping between bettmensch.ai pipelines and ArgoWorkflow resources:

| bettmensch.ai | ArgoWorkflow      |
| --------------|-------------------|
| `Flow`        | `WorkflowTemplate`|
| `Run`         | `Workflow`        |

## Define script templates

In [14]:
from hera.workflows import script, Parameter, Artifact, models as m
from pathlib import Path

@script(outputs=[Artifact(name="product",path="product")],)
def multiply(a: float, b: float):
    
    with open('product','w') as output:
        output.write(str(a * b))
        
@script(inputs=[Artifact(name='artifact_i',path="artifact_i")])
def artifact_io():
    
    import os
    print(os.getcwd())
    print(os.listdir(os.getcwd()))
    
    with open('artifact_i','r') as a_file:
        art_in = a_file.read()

## Define Workflow and DAG

In [15]:
from hera.workflows import WorkflowTemplate, DAG, Parameter, S3Artifact
from hera.workflows.models import ArtifactRepositoryRef

dag_name = "s3-artifact-io"

with WorkflowTemplate(
    generate_name="s3-artifact-io-", 
    entrypoint=dag_name,
    namespace="argo",
    arguments=[Parameter(name="a"),Parameter(name="b")],
    # the Workflow referencing this template inherits the `spec` level `workflow_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    #artifact_repository_ref={"configmap":"bettmensch-ai-artifact-repository","key":"bettmensch-ai-artifact-repository"}
    ) as wt:
    
    wt_a, wt_b, = wt.get_parameter('a').value,wt.get_parameter('b').value 
    
    with DAG(name=dag_name) as dag:
        
        a_times_b = multiply(
            name="multiply-a-and-b",arguments=[
                Parameter(name="a",value=wt_a),
                Parameter(name="b",value=wt_b)
            ]
        )
        artifact_io = artifact_io(
            name="artifact-io",
            arguments=[
                a_times_b.get_artifact('product').with_name("artifact_i"),
            ],    
        )
        a_times_b >> artifact_io 

In [16]:
wt.to_file('.')

PosixPath('/home/ubuntu/repositories/bettmensch.ai/sdk/notebooks/s3-artifact-io.yaml')

In [17]:
wt.create()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


WorkflowTemplate(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2024, 7, 8, 17, 53, 2, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='s3-artifact-io-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2024, 7, 8, 17, 53, 2, tzinfo=datetime.timezone.utc)))], name='s3-artifact-io-l2b98', namespace='argo', owner_references=None, resource_version='101004', self_link=None, uid='6861de68-96c0-44bb-b42f-968ef8362a02'), spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=[Parameter(default=None, descr

In [69]:
artifact = a_times_b.get_artifact('product').with_name("artifact_i")

In [72]:
artifact.__dict__

{'name': 'artifact_i',
 'archive': None,
 'archive_logs': None,
 'artifact_gc': None,
 'deleted': None,
 'from_': '{{tasks.multiply-a-and-b.outputs.artifacts.product}}',
 'from_expression': None,
 'global_name': None,
 'mode': None,
 'path': None,
 'recurse_mode': None,
 'sub_path': None,
 'loader': None,
 'optional': None,
 'output': False}

In [73]:
type(artifact)

hera.workflows.artifact.Artifact